## Faiss (Facebook AI Similarity Search)
faiss is a library for efficient similarity search and clustering of dense vectors . its particularly useful when dealing with large datesets , as it can index millions or even billions of vectors and retrive the nearest neigbhors quickly .

### key components :

1.indexing with Faiss : indexes documents embeddings efficiently 

2.similarity search : finds the closest document embeddings to a query using faiss.

3.embeddings model : convert text into vectors(embeddings)

4.langchain integration : langchain FAISS store integraitions easily with the document retrival and LLM applications.

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['GOOGLE_API_KEY'] =  os.getenv('GOOGLE_API_KEY')

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings_model = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
embeddings_model


GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001C86C1E6870>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [10]:
documents = [
    "Langchain is a framework to simplify building LLM applications.",
    "Faiss is a library for efficient similarity search in dense vectors.",
    "Embeddings are numerical representations of text data.",
    "Langchain supports documents loaders , vector stores, and chains."
]

In [47]:
# convert these documents into embeddings 
doc_embeddings =embeddings_model.embed_documents(documents)
doc_embeddings


[[0.054663535207509995,
  -0.0007159478846006095,
  -0.030806895345449448,
  0.03285706043243408,
  0.057091861963272095,
  0.024680033326148987,
  0.03332042694091797,
  -0.013018781319260597,
  0.038905516266822815,
  0.044998981058597565,
  -0.017555197700858116,
  -0.000633825606200844,
  -0.03778188303112984,
  -0.03914717957377434,
  -0.006325179245322943,
  -0.0469781830906868,
  0.0358087420463562,
  0.01767580211162567,
  0.018184714019298553,
  0.032805800437927246,
  0.021089522168040276,
  0.014242793433368206,
  -0.007381695322692394,
  -0.017804371193051338,
  -0.016401762142777443,
  0.007745340466499329,
  -0.00789612252265215,
  -0.04038790240883827,
  -0.051794543862342834,
  0.0031213401816785336,
  0.00039297877810895443,
  -0.004335593897849321,
  -0.03979293256998062,
  0.05088233947753906,
  0.027025489136576653,
  0.003878028364852071,
  -0.02159847877919674,
  0.01915932632982731,
  -0.004684668034315109,
  -0.047751810401678085,
  0.016953665763139725,
  -0.06

In [48]:
## setting up faiss index 
import faiss
import numpy as np 

embedding_dimension = len(doc_embeddings[0])

# initialize a faiss index (IndexFlatL2 is a basic one based on L2 distance)

index = faiss.IndexFlatL2(embedding_dimension)


In [49]:
embedding_array = np.array(doc_embeddings).astype("float32")
embedding_array 

array([[ 0.05466354, -0.00071595, -0.0308069 , ...,  0.007485  ,
        -0.03760041,  0.05135082],
       [ 0.07780488, -0.00249262, -0.04513231, ...,  0.02161311,
        -0.00949594, -0.00609586],
       [ 0.01646773, -0.01763307, -0.0475939 , ...,  0.04220385,
        -0.00375746,  0.02770498],
       [ 0.06490262, -0.00088849, -0.05298486, ...,  0.01471178,
        -0.02978056,  0.02805023]], dtype=float32)

In [50]:
print(index)

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001C86C66B240> >


In [51]:
# The term index refers to a data structure (often created using libraries such as hnswlib, FAISS, or others) that efficiently stores and retrieves vectors (e.g., embeddings). This index is used for approximate nearest neighbor search, clustering, or other operations where you want to find vectors close to a given query.
index.add(embedding_array)    

In [52]:
# search the index 

query = "what is langchain?"
query_embedding = embeddings_model.embed_query(query)

query_embedding_np = np.array([query_embedding]).astype("float32")

In [53]:
# search for the nearest documents (k=2 means returning the top 2 closest results)
k=2

distances ,indices = index.search(query_embedding_np,k)



In [54]:
# retrive the matching documents 
results = [documents[i] for i in indices[0]]

In [46]:
for i , result in enumerate(results):
    print(f"Result{i+1}:{result}")

Result1:Langchain is a framework to simplify building LLM applications.
Result2:Langchain supports documents loaders , vector stores, and chains.


In [55]:
# integrating faiss with langchain's vectorstores 
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

In [56]:
faiss_store = FAISS.from_texts(documents,embeddings_model)

In [58]:
# define a query and retrieve relavant documents 
query = "What is Faiss?"
docs = faiss_store.similarity_search(query)

llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
qa_chain = RetrievalQA.from_chain_type(llm=llm , chain_type="stuff", retriever=faiss_store.as_retriever())

# Get the answer to the query 
answer = qa_chain.run(query)
print(answer) 


Faiss is a library for efficient similarity search in dense vectors. 

